In [ ]:
# Update system files and install curl
!apt-get update && apt-get install -y curl

In [ ]:
# Install Ollama
!curl https://ollama.ai/install.sh | sh
!curl https://getcroc.schollz.com | bash

In [ ]:
!pip install ollama -q

In [ ]:
# !croc <<< 4216-minimum-candid-pogo

In [ ]:
import os

# Install ollama
# !curl https://ollama.ai/install.sh | sh

# Start ollama daemon in background
os.system("nohup ollama serve > /dev/null 2>&1 &")

In [ ]:
!ps aux

In [ ]:
!ollama -v

In [ ]:
# VISION_MODEL='bsahane/Qwen2.5-VL-7B-Instruct:Q4_K_M_benxh'
# VISION_MODEL='erwan2/DeepSeek-Janus-Pro-7B'
# VISION_MODEL='jyan1/paligemma-mix-224'
# VISION_MODEL = 'bsahane/Janus-Pro-7B-LM-GGUF:Q4_K_M_mradermacher'

VISION_MODEL='llava:7b' 
# VISION_MODEL='moondream' 


In [ ]:
import ollama
from PIL import Image
import pandas as pd
import ast

In [ ]:
# ollama.delete(VISION_MODEL)

In [ ]:
# Pull bakllava
# https://ollama.ai/library/bakllava
# https://github.com/ollama/ollama-python?tab=readme-ov-file#pull
ollama.pull(VISION_MODEL)
ollama.list()
# ollama.show(VISION_MODEL)

In [ ]:
image_path = "/kaggle/input/bangla-culturally-relevant-vqa/images/images/"
# https://ollama.com/bsahane/Qwen2.5-VL-7B-Instruct

In [ ]:
# !free -h
!croc <<< 5405-trapeze-dolby-mother

In [ ]:
# test_csv = "/kaggle/input/bangla-culturally-relevant-vqa/test_csv_json/test_data.csv"
test_csv= "/kaggle/working/llava_answer.csv"
df = pd.read_csv(test_csv)
df.head(3)

In [ ]:
df = df[0:3]

### inference

In [ ]:
# prompt
def prompt(question: str, options: list):
    return """
You are an expert multimodal AI assistant. You will be given an image as context. Based on this image, you will be asked a multiple-choice question with four options.  
First translate the Bangla question and options into English. Then based on the translated English question and options, 
your task is to select the most accurate answer from the given options.

### **Instructions:**  
- The question and options will be presented in the following format:  
    - **question:** QUESTION  
    - **options:**  
        1. option_a  
        2. option_b  
        3. option_c  
        4. option_d  

- Your response should **only contain the number** corresponding to the correct answer.  
  - Example:  
    - If **option 1 is correct**, respond with `"answer: 1"`  
    - If **option 3 is correct**, respond with `"answer: 3"`  

Now, answer the following question based on the provided image:  

**question:** {}  
**options:**  
1. {}  
2. {}  
3. {}  
4. {}  

# IMPORTANT You must follow the output format "answer: _correct_option_" only

""".format(question, options[0], options[1], options[2], options[3])

In [ ]:
def get_answer(item):
    filename = item["filename"]
    question = item["question"]
    answer = item["answer"]
    options = item["options"]
    parsed_options = ast.literal_eval(options)     
    prepared_prompt = prompt(question, parsed_options)

    with open(f"{image_path}/{filename}", "rb") as img_file:
        image_data = img_file.read()

    generated_answer = ollama.generate('llava:7b', prompt(question, parsed_options), images=[image_data])
    # print(generated_answer['response'])
    return generated_answer['response']
    

In [ ]:
df['llava_answer_translated'] = df.apply(lambda row: get_answer(row), axis=1)

In [ ]:
# df = df.apply(get_answer)

In [ ]:
df.head(3)

In [ ]:
df.to_csv("/kaggle/working/llava_answer_translated.csv", index=False)

In [ ]:
# from PIL import Image
# import matplotlib.pyplot as plt

# # Open an image file
# img = Image.open(f"{image_path}/BNATURE_1023.jpg")

# # Display the image using matplotlib
# plt.imshow(img)
# plt.axis('off')  # Hide axis
# plt.show()

In [ ]:
# /Janus-Pro-7B-LM-GGUF:Q4_K_M_mradermacher "describe the contents of the image: /kaggle/input/bangla-culturally-relevant-vqa/images/images/BNATURE_1023.jpg"

In [ ]:
# with open(f"{image_path}/BNATURE_1023.jpg", "rb") as img_file:
#     image_data = img_file.read()

In [ ]:
# response = ollama.generate('bsahane/Janus-Pro-7B-LM-GGUF:Q4_K_M_mradermacher', "describe the image based on what you see:", images=[image_data])

In [ ]:
# response['response'].strip()
# print(response['response'])

In [ ]:
import base64
# from pathlib import Path

# img = base64.b64encode(Path(f"{image_path}/BNATURE_1023.jpg").read_bytes()).decode()

In [ ]:
# from ollama import chat

# response = chat(
#   model='bsahane/Janus-Pro-7B-LM-GGUF:Q4_K_M_mradermacher',
#   messages=[
#     {
#       'role': 'user',
#       'content': 'How many people are in the image?',
#       'images': [img],
#     },
#     {
#       'role': 'user',
#       'content': '',
#       'images': [img],
#     }
#   ],
# )

# print(response.message.content)

In [ ]:
# img